## READ FIRST

### This document is organized as follows:
#### 1. Required install
#### 2. Libraries used throughout the document
#### 3. Questions, comments and results

#### Everything was run using JupyterLab version: and Python version: 3.6.8 on MacOS
#### if you want to run this notebook, be sure to have all the related files (webdriver) in the same folder as this file. 




#### Install

In [ ]:
#pip install selenium
#pip install webdriver-manager

#### Libraries 

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd 
import numpy as np

## Question 1: 

In [8]:
#install webdriver. I had some compatibility issues hence I'm using the webdriver w/installation at the same time 
driver = webdriver.Chrome(ChromeDriverManager().install())

#go to the requested URL
driver.get('https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE14333')



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/luciananieto/.wdm/drivers/chromedriver/mac64/94.0.4606.41/chromedriver] found in cache


In [9]:
#in order to "see" all of the elements in the page, both the three already displayed and the ones that can be accessed though +More
all_elements = driver.find_elements_by_tag_name('div')
for element in all_elements:
    property = element.value_of_css_property('display')
    if property == 'none':
        driver.execute_script(
        "arguments [0].style.display = 'block';", element)
    if property == 'block':
        driver.execute_script(
        "arguments[0].style.display = 'none';", element)


In [10]:
#once all the elements are located we can extract the xpath, in this examples we separated it in two strings (s1, s2) to avoid a long string

s1 = '//*[@id="ui-ncbiexternallink-1"]/table/tbody/tr/td/table[6]/tbody/tr[3]/td[2]/'
s2 ='table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/table[1]/tbody/tr[28]/td[2]'
table= driver.find_element_by_xpath(s1+s2)
tbody = table.find_elements_by_tag_name('tbody')


#create an empty list to store the links. Locate the elements by tag (a), and then retrieve the corresponding link, stored within the 'href' attribute
#append each link to the list
links =[]
b1=tbody[0]
f3=b1.find_elements_by_tag_name('a')
for a in f3:
    links.append( a.get_attribute('href') )
b2 =tbody[1]
all_a= b2.find_elements_by_tag_name('a')
for a in all_a:
    links.append( a.get_attribute('href'))
    

#create an empty list to store the ids for each sample after extracting them from each corresponding URL
id_ =[]
for link in links:
    t = link.split('/acc.cgi?acc=')[-1]
    id_.append(t)


### Final output for question 1

In [11]:
#create a dictionary with the id as key and the links as values; print the dictionary as txt if desire or simply print it to the console

links_dict = {}
for key in id_:
    for value in links:
        links_dict[key]= value
        links.remove(value)
        break
        
print(links_dict)

with open('links_dict.txt', 'w') as f:
    print(links_dict, file=f)

{'GSM358341': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358341', 'GSM358342': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358342', 'GSM358343': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358343', 'GSM358344': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358344', 'GSM358345': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358345', 'GSM358346': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358346', 'GSM358347': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358347', 'GSM358348': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358348', 'GSM358349': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358349', 'GSM358350': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358350', 'GSM358351': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358351', 'GSM358352': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358352', 'GSM358353': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM358353'

## Question 2 

In [12]:
#create an empty list to store the micro array data from each sample 
#this for loop will grab each link from the dictiorary greated before,
#will wait for 10 sec till the full page is loaded
#find the link text for fulltable, and clik to open a new tab, 
#wait until two tabs are open and switch the driver to the new one 
#retrieve the text within the window 
#store the text in the list 
#close the tab, go back to the original, repreat for all links in link_dict
#/html/body/font/pre
micro= []

for link in links_dict.values():
    # Open URL
    driver.get(link)

    # Setup wait for later
    wait = WebDriverWait(driver, 10)

    # Set handle windows
    driver.current_window_handle

    # Click the link which opens in a new window
    driver.find_element_by_name("fulltable").click()

    # Wait for the new window or tab
    wait.until(EC.number_of_windows_to_be(2))
    
    #switch to the new tab
    driver.switch_to.window(driver.window_handles[1])
    
    #find element by xpath
    s= driver.find_element_by_xpath("/html/body/font/pre").text
    
    #append the data to the list 
    micro.append(s)
    
    #close driver in the tab 
    driver.close()
    
    #return the driver to the previous window 
    driver.switch_to.window(driver.window_handles[0])



### Final output question 2

In [13]:
#save the result as CSV
np.savetxt("micro_b.csv", 
           micro,
           delimiter ="_", 
           fmt ='% s')

## Question 3

In [14]:
#open the file to clean and remove the unnecessary data 
  
# Read the csv file
df = pd.read_csv('micro_b.csv') 
  
#drop null values 
df2 = df.dropna(how = 'all') 

#check df shape for later comparison 
print(df.shape)

(15856619, 1)


In [15]:
#drop and store as new dfs 
df3=df2.drop(df2[df2['#ID_REF = '] =='ID_REF VALUE'].index)
df4=df3.drop(df3[df3['#ID_REF = '] =='#VALUE = log-base-2 values of quantile normalized MAS5.0 calls'].index)

In [16]:
#df4.shape

In [17]:
#this step was used to separate the ID_REF from the values, and to create a column as comparison to check if this was ok 
df_new = df4.rename(columns={'#ID_REF = ': 'test'})

In [18]:
#df_new.columns

In [8]:
#run at your own risk!!!!! it took me 3 hs to run this cell 
#this part is performing an operation over more than 15M rows it worked, but I guess it's a better and more efficient way to perform this 

df_new[['ID_REF','VALUE']] = df_new.test.apply(
lambda x: pd.Series(str(x).split("_at ")))

df_new.head()


,test,ID_REF,VALUE
2,1007_s_at 9.947789711,1007_s,9.947789711
3,1053_at 8.051414734,1053,8.051414734
4,117_at 5.754467818,117,5.754467818
5,121_at 8.789127147,121,8.789127147
6,1255_g_at 4.774139199,1255_g,4.774139199


In [19]:
#save as checkpoint
df_new.to_csv('micro_ok.csv',index=False)

In [16]:
#this was performed because some extra cells appeared under the name "#ID_REF"
df_new=df_new.drop(df_new[df_new['ID_REF'] =='#ID_REF = '].index)
df_new.shape

(15855750, 3)

In [17]:
#to group the values for each sample (290) according to the micro names (~54K)
grouped_df = df_new.groupby("ID_REF")
grouped_lists = grouped_df["VALUE"].apply(list)
grouped_lists.head()

ID_REF
1007_s    [9.947789711, 9.951618512, 10.13171984, 10.586...
1053      [8.051414734, 7.36085168, 8.343533699, 7.78866...
117       [5.754467818, 7.391554385, 5.889777655, 6.4151...
121       [8.789127147, 8.816099466, 8.827682083, 9.7391...
1255_g    [4.774139199, 4.716702126, 4.27842389, 4.46106...
Name: VALUE, dtype: object

In [31]:
#transform this into a df 
df_290= grouped_lists.apply(lambda x: pd.Series(list(x))) \
    .rename(columns=lambda x: 'ph{}'.format(x + 1))

### Final output question 3

In [122]:
#append the id names from each sample as column names for each value (290)
#id_ = values stored as a list in question 1 
my_columns = id_
df_290.columns = my_columns
df_290.to_csv('micro_array2.csv', index=True)

## Question 5

In [20]:
#open the previously crated df with the micro array data 
df_micro = pd.read_csv('micro_array2.csv') 

#calcualte the absolute value for each value in the df, without counting the first column
df_abs = df_micro[df_micro.columns[1:291]].abs()


In [21]:
#using nlargest find the highest 5 absolute value per row in the df
#this method creates a new object where only the top 5 values are shown and the rest in presented as NaN
t= df_abs.apply(pd.Series.nlargest, axis=1, n=5)

In [22]:
t.head(6)

,GSM358341,GSM358342,GSM358343,GSM358344,GSM358345,GSM358346,GSM358347,GSM358348,GSM358349,GSM358350,...,GSM358621,GSM358622,GSM358623,GSM358624,GSM358625,GSM358626,GSM358627,GSM358628,GSM358629,GSM358630
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,8.657586,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.949735,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,9.739125,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#since only the top 5 elements are present now in the object we can apply .mean(), per row (axis=1),
#and store that as a new column call "mean"
t['mean']=t.mean(axis=1)

In [24]:
#we select that column and later crate a copy to concatenate it to the previous df with all the information 
selected_columns = t[["mean"]]

In [25]:
mean =selected_columns.copy()

In [26]:
df_micro.head(2)

,ID_REF,GSM358341,GSM358342,GSM358343,GSM358344,GSM358345,GSM358346,GSM358347,GSM358348,GSM358349,...,GSM358621,GSM358622,GSM358623,GSM358624,GSM358625,GSM358626,GSM358627,GSM358628,GSM358629,GSM358630
0,1007_s,9.947790,9.951619,10.131720,10.586761,10.407607,10.149212,10.148531,10.310156,9.393782,...,8.666636,10.396283,9.958790,10.122036,9.91746,9.780188,10.495631,9.998563,10.462451,9.981964
1,1053,8.051415,7.360852,8.343534,7.788664,8.657586,8.040645,8.041764,7.627312,8.142250,...,6.916529,7.758786,8.148096,7.839722,7.46405,7.678973,8.949735,8.355339,8.164194,7.805399


In [27]:
#mean.head(2)

In [28]:
#for the final df we can join both dfs the one with all the information (df_micro) and the new one with only the mean for each row "mean"
final = pd.concat([df_micro, mean], axis=1, join='outer')

In [29]:
final.head(2)

,ID_REF,GSM358341,GSM358342,GSM358343,GSM358344,GSM358345,GSM358346,GSM358347,GSM358348,GSM358349,...,GSM358622,GSM358623,GSM358624,GSM358625,GSM358626,GSM358627,GSM358628,GSM358629,GSM358630,mean
0,1007_s,9.947790,9.951619,10.131720,10.586761,10.407607,10.149212,10.148531,10.310156,9.393782,...,10.396283,9.958790,10.122036,9.91746,9.780188,10.495631,9.998563,10.462451,9.981964,10.766383
1,1053,8.051415,7.360852,8.343534,7.788664,8.657586,8.040645,8.041764,7.627312,8.142250,...,7.758786,8.148096,7.839722,7.46405,7.678973,8.949735,8.355339,8.164194,7.805399,8.887170


In [30]:
#to sort the values first based on the absolute mean and then by ID without loosing the previous sort we need to perform this together in the desired order,
#sorting in descending order
final.sort_values(by=['mean', 'ID_REF'], ascending=False, inplace=True)


In [31]:
final.head()

,ID_REF,GSM358341,GSM358342,GSM358343,GSM358344,GSM358345,GSM358346,GSM358347,GSM358348,GSM358349,...,GSM358622,GSM358623,GSM358624,GSM358625,GSM358626,GSM358627,GSM358628,GSM358629,GSM358630,mean
54673,AFFX-r2-P1-cre-3,13.173938,12.934006,12.733876,12.780998,12.608079,12.791955,12.770589,13.005125,12.842837,...,12.795980,12.988141,12.959562,12.929165,13.091828,13.103858,12.959562,12.889895,12.918704,13.271998
35387,226131_s,13.205576,13.054339,13.205576,13.152005,13.173938,13.173938,13.091828,12.871847,13.271998,...,12.780998,12.663308,12.859210,12.847055,12.834713,12.704966,12.880923,12.791955,12.970338,13.271998
24415,215121_x,12.838890,13.152005,13.005125,10.306627,13.205576,12.516588,13.152005,12.976111,11.192040,...,12.711592,6.309420,11.707237,12.582268,11.402725,10.866322,10.215909,9.214688,12.842837,13.271998
20031,210646_x,13.117366,12.993441,13.080976,13.080976,13.152005,13.132073,13.038620,13.103858,12.964846,...,12.988141,12.588615,12.909510,12.923526,13.011582,12.780998,12.913893,12.954997,12.923526,13.271998
14339,204892_x,13.152005,13.103858,12.964846,13.062872,13.011582,13.031634,13.071947,13.011582,13.062872,...,12.885603,12.757188,12.819010,12.899878,12.909510,12.949391,12.815276,12.764179,12.774037,13.271998


### Final output question 5

In [32]:
#create a new df with only the columns of interest 
id_mean = final[["ID_REF", "mean"]]
id_mean2 =id_mean.copy()

In [33]:
#create a df with only the first 3 rows 
id_mean3= id_mean2.iloc[[0,1,2]]

In [34]:
#check if it is correct 
id_mean3

,ID_REF,mean
54673,AFFX-r2-P1-cre-3,13.271998
35387,226131_s,13.271998
24415,215121_x,13.271998


In [35]:
#store the result as a dict 
highest_three = id_mean3.set_index('ID_REF').T.to_dict('list')

In [36]:
highest_three

{'AFFX-r2-P1-cre-3': [13.27199773],
 '226131_s': [13.27199773],
 '215121_x': [13.27199773]}

In [37]:
#check data type
type(highest_three)

dict

In [38]:
#I added this part to add the "_at" removed in the process of separating the ids from the values 
#I'm sure there is a better way to do it 

def Convert(lst2):
    res_dct2 = {lst2[i]: lst2[i + 1] for i in range(0, len(lst2), 2)}
    return res_dct2
         
# Driver code
lst2 = ['AFFX-r2-P1-cre-3_at' , 13.27199773,
  '226131_s_at', 13.27199773,
  '215121_x_at', 13.27199773]
highest_three=Convert(lst2)

In [39]:
type(highest_three)

dict

## Question 6

In [40]:
#we need to call again the driver and open the URL 

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE14333')



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/luciananieto/.wdm/drivers/chromedriver/mac64/94.0.4606.41/chromedriver] found in cache


In [41]:
#once we found the specidfied component in the webpage and after inspecting it we copy the xpath
#once again stored as a two separeted strings only for the purpouses of aesthetics 
#merged under the name "long_path"

a1 = '//*[@id="ui-ncbiexternallink-1"]/table/tbody/tr/td/table[6]/tbody/tr[3]/td[2]/table/tbody/tr/td/'
a2 ='table/tbody/tr/td/table[2]/tbody/tr/td/table[1]/tbody/tr[17]/td[2]'
long_path = (a1+a2)

In [42]:
#create an empty list to store the values from each sample 
#this for loop will grab each link from the dictiorary greated before,
#will wait for 10 sec till the full page is loaded
#find the element in the xpath definded by long_path,
#retrieve the text within the window 
#store the text in the list 
#close the tab, go back to the original, repreat for all links in link_dict
#then the driver will close the window and quit 

char= []

for link in links_dict.values():

    driver.get(link)

    wait = WebDriverWait(driver, 10)

    driver.current_window_handle
    c= driver.find_element_by_xpath(long_path).text
    char.append(c)
    
driver.close()
driver.quit


<bound method WebDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="fa54ef53adef32355a05dbd99f228dbb")>>

In [43]:
#we save the information as a csv
np.savetxt("charact.csv", 
           char,
           delimiter ="_", 
           fmt ='% s')

In [44]:
#open the new file and add a name for the header 
charact = pd.read_csv('charact.csv',names=["id"]) 

In [45]:
charact.head()

,id
0,Location: Right; DukesStage: A; Age_Diag: 78; ...
1,Location: Rectum; DukesStage: A; Age_Diag: 53;...
2,Location: Left; DukesStage: A; Age_Diag: 80; G...
3,Location: Left; DukesStage: A; Age_Diag: 58; G...
4,Location: Left; DukesStage: A; Age_Diag: 81; G...


In [46]:
#create new columns based on the names of each characterist and 4 more for the rest of the data (later we will drop these)
#we will split the column in more columns based on "; " (notice the space after the ; )
charact[["Location", "DukesStage","Age_Diag","Gender", "other1",'other2','other3','other4']] = charact.id.str.split("; ",expand=True,)
#charact

In [47]:
#now we are dropping the columns not used in this study  
charact.drop(['id',"other1",'other2','other3','other4'], axis=1, inplace=True)

In [48]:
charact.head()

,Location,DukesStage,Age_Diag,Gender
0,Location: Right,DukesStage: A,Age_Diag: 78,Gender: M
1,Location: Rectum,DukesStage: A,Age_Diag: 53,Gender: F
2,Location: Left,DukesStage: A,Age_Diag: 80,Gender: F
3,Location: Left,DukesStage: A,Age_Diag: 58,Gender: M
4,Location: Left,DukesStage: A,Age_Diag: 81,Gender: M


In [49]:
#althoug the df contains all the information still each cell contaings the name of the characterist, to remove this:
#for each column we are searching for the string to remove (text after lstrip), 
#NOTE: be sure to add the space after the name: , otherwise the space will be part of the value remaining in the cell  

charact['Location'] = charact['Location'].map(lambda x: x.lstrip('Location: '))
charact['DukesStage'] = charact['DukesStage'].map(lambda x: x.lstrip('DukesStage: '))
charact['Age_Diag'] = charact['Age_Diag'].map(lambda x: x.lstrip('Age_Diag: '))
charact['Gender'] = charact['Gender'].map(lambda x: x.lstrip('Gender: '))

### Final output question 6 

In [50]:
#save this as a CSV
charact.to_csv('characteristics.csv', index=True)

## Question 7

In [51]:
charact.shape

(290, 4)

In [52]:
#to calcualte the mode of the df per column simply .mode()
variables_mode = charact.mode()

In [53]:
#check if it was ok 
variables_mode

,Location,DukesStage,Age_Diag,Gender
0,Right,B,63,M


### Final output question 7

In [54]:
#crate a dictionary with the data 

def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct
         
# Driver code
lst = ['Location', 'Right',
  'DukesStage', 'B',
  'Age_Diag', 63.0,
  'Gender', 'M']
variables_mode=Convert(lst)

In [55]:
variables_mode

{'Location': 'Right', 'DukesStage': 'B', 'Age_Diag': 63.0, 'Gender': 'M'}

# for gradescope 

In [56]:
result = {'links': links_dict,
          'highest_three':highest_three,
          'mode': variables_mode
         }
import json
result_str= json.dumps(result)
textfile = open('result.txt', 'w')
textfile.write(result_str)
textfile.close()